### Installing Required Packages

In [ ]:
!pip install -q git+https://github.com/One-sixth/fairseq.git sacrebleu tensorboardX

### Pipeline C (Morfessor, 10k BPE Operations)

#### Preprocessing Data

In [ ]:
%cd /content/drive/MyDrive/NLP_TRANSLATION
!mkdir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/bin
!mkdir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/checkpoints

In [ ]:
!fairseq-preprocess \
    --source-lang fi --target-lang en \
    --trainpref /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/data/train.morf \
    --validpref /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/data/dev.morf \
    --testpref /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/data/test.morf \
    --destdir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/bin \
    --workers 20

#### Training Fairseq

In [ ]:
!fairseq-train /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/bin \
    --arch transformer_iwslt_de_en \
    --share-decoder-input-output-embed \
    --source-lang fi --target-lang en \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8192 \
    --update-freq 1 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok space \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/checkpoints \
    --max-epoch 30 \
    --patience 10 \
    --fp16 \
    --no-epoch-checkpoints

#### Testing Fairseq

In [ ]:
!fairseq-generate /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/bin \
    --path /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5 \
    --remove-bpe \
    > /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_10k_@@/results_C_10k_@@.txt

#### Evaluation (SacreBLEU)


> _NOTE:_  Be sure you are under the Project Root Folder (`NLP_TRANSLATION`).

In [ ]:
%%bash
moses_scripts="tools/mosesdecoder/scripts"

input_file="pipeline_C_10k_@@/results_C_10k_@@.txt"

original_ref_file="data/test/test.en"

# temporary files
src_tok="pipeline_C_10k_@@/source.tok"
hypo_tok="pipeline_C_10k_@@/hypothesis.tok"
ref_tok="pipeline_C_10k_@@/reference.tok"

src_dtc="pipeline_C_10k_@@/source.dtc"
hypo_dtc="pipeline_C_10k_@@/hypothesis.dtc"
ref_dtc="pipeline_C_10k_@@/reference.dtc"

# output files
src_detok="pipeline_C_10k_@@/source.detok"
hypo_detok="pipeline_C_10k_@@/hypothesis.detok"
ref_detok="pipeline_C_10k_@@/reference.detok"

# extract detokenized texts
grep ^S- "$input_file" | sed 's/^S-//' | sort -n | cut -f2- > "$src_tok"
grep ^H- "$input_file" | sed 's/^H-//' | sort -n | cut -f3- > "$hypo_tok" # hypothesis (translated text)
grep ^T- "$input_file" | sed 's/^T-//' | sort -n | cut -f2- > "$ref_tok" # refernece (gold standard text)

# detruecasing
perl "$moses_scripts/recaser/detruecase.perl" < "$src_tok" > "$src_dtc"
perl "$moses_scripts/recaser/detruecase.perl" < "$hypo_tok" > "$hypo_dtc"
perl "$moses_scripts/recaser/detruecase.perl" < "$ref_tok" > "$ref_dtc"

# detokenizing
perl "$moses_scripts/tokenizer/detokenizer.perl" -l fi < "$src_dtc" > "$src_detok"
perl "$moses_scripts/tokenizer/detokenizer.perl" -l en < "$hypo_dtc" > "$hypo_detok"
perl "$moses_scripts/tokenizer/detokenizer.perl" -l en < "$ref_dtc" > "$ref_detok"

score=$(sacrebleu "$original_ref_file" -i "$hypo_detok" -m bleu -b -w 4)

# cleanup
rm "$src_tok" "$hypo_tok" "$ref_tok" "$src_dtc" "$hypo_dtc" "$ref_dtc"
echo "Pipeline C (10k) BLEU SCORE: $score"

### Pipeline C (Morfessor, 20k BPE Operations)

#### Preprocessing Data

In [ ]:
%cd /content/drive/MyDrive/NLP_TRANSLATION
!mkdir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/bin
!mkdir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/checkpoints

In [ ]:
!fairseq-preprocess \
    --source-lang fi --target-lang en \
    --trainpref /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/data/train.morf \
    --validpref /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/data/dev.morf \
    --testpref /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/data/test.morf \
    --destdir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/bin \
    --workers 20

#### Training Fairseq

In [ ]:
!fairseq-train /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/bin \
    --arch transformer_iwslt_de_en \
    --share-decoder-input-output-embed \
    --source-lang fi --target-lang en \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 8192 \
    --update-freq 1 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok space \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/checkpoints \
    --max-epoch 30 \
    --patience 10 \
    --fp16 \
    --no-epoch-checkpoints

#### Testing Fairseq

In [ ]:
!fairseq-generate /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/bin \
    --path /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/checkpoints/checkpoint_best.pt \
    --batch-size 128 \
    --beam 5 \
    --remove-bpe \
    > /content/drive/MyDrive/NLP_TRANSLATION/pipeline_C_20k_@@/results_C_20k_@@.txt

#### Evaluation (SacreBLEU)


> _NOTE:_  Be sure you are under the Project Root Folder (`NLP_TRANSLATION`).

In [ ]:
%%bash
moses_scripts="tools/mosesdecoder/scripts"

input_file="pipeline_C_20k_@@/results_C_20k_@@.txt"

original_ref_file="data/test/test.en"

# temporary files
src_tok="pipeline_C_20k_@@/source.tok"
hypo_tok="pipeline_C_20k_@@/hypothesis.tok"
ref_tok="pipeline_C_20k_@@/reference.tok"

src_dtc="pipeline_C_20k_@@/source.dtc"
hypo_dtc="pipeline_C_20k_@@/hypothesis.dtc"
ref_dtc="pipeline_C_20k_@@/reference.dtc"

# output files
src_detok="pipeline_C_20k_@@/source.detok"
hypo_detok="pipeline_C_20k_@@/hypothesis.detok"
ref_detok="pipeline_C_20k_@@/reference.detok"

# extract detokenized texts
grep ^S- "$input_file" | sed 's/^S-//' | sort -n | cut -f2- > "$src_tok"
grep ^H- "$input_file" | sed 's/^H-//' | sort -n | cut -f3- > "$hypo_tok" # hypothesis (translated text)
grep ^T- "$input_file" | sed 's/^T-//' | sort -n | cut -f2- > "$ref_tok" # refernece (gold standard text)

# detruecasing
perl "$moses_scripts/recaser/detruecase.perl" < "$src_tok" > "$src_dtc"
perl "$moses_scripts/recaser/detruecase.perl" < "$hypo_tok" > "$hypo_dtc"
perl "$moses_scripts/recaser/detruecase.perl" < "$ref_tok" > "$ref_dtc"

# detokenizing
perl "$moses_scripts/tokenizer/detokenizer.perl" -l fi < "$src_dtc" > "$src_detok"
perl "$moses_scripts/tokenizer/detokenizer.perl" -l en < "$hypo_dtc" > "$hypo_detok"
perl "$moses_scripts/tokenizer/detokenizer.perl" -l en < "$ref_dtc" > "$ref_detok"

score=$(sacrebleu "$original_ref_file" -i "$hypo_detok" -m bleu -b -w 4)

# cleanup
rm "$src_tok" "$hypo_tok" "$ref_tok" "$src_dtc" "$hypo_dtc" "$ref_dtc"
echo "Pipeline C (20k) BLEU SCORE: $score"

---